# US Inequality Atlas -- Exploration Notebook

**County-level data on food deserts, housing crisis, healthcare deserts, veterans, and hospitals**

Author: [Luke Steuber](https://lukesteuber.com) | Bluesky: [@lukesteuber.com](https://bsky.app/profile/lukesteuber.com)

Dataset: [lukeslp/us-inequality-atlas](https://huggingface.co/datasets/lukeslp/us-inequality-atlas)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import json

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Load all CSV datasets
food = pd.read_csv('food_deserts/food_desert_merged.csv')
housing = pd.read_csv('housing/housing_crisis_merged.csv')
healthcare = pd.read_csv('healthcare/healthcare_desert_merged.csv')
veterans = pd.read_csv('veterans/military_firearm_merged_analysis.csv')
hospitals = pd.read_csv('cms/cms_hospitals_20260121.csv')

print("Datasets loaded:")
print(f"  Food deserts:      {len(food):>6,} counties, {len(food.columns)} columns")
print(f"  Housing crisis:    {len(housing):>6,} counties, {len(housing.columns)} columns")
print(f"  Healthcare deserts:{len(healthcare):>6,} counties, {len(healthcare.columns)} columns")
print(f"  Veterans/firearms: {len(veterans):>6,} states, {len(veterans.columns)} columns")
print(f"  CMS hospitals:     {len(hospitals):>6,} facilities, {len(hospitals.columns)} columns")

## Food Deserts -- Poverty and Food Access

County-level analysis of poverty rates, SNAP eligibility, and vehicle access.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Poverty rate distribution
axes[0].hist(food['poverty_rate'].dropna(), bins=50, color='#F44336', edgecolor='white', alpha=0.8)
axes[0].set_xlabel('Poverty Rate (%)')
axes[0].set_ylabel('Number of Counties')
axes[0].set_title('County Poverty Rate Distribution')
axes[0].axvline(food['poverty_rate'].median(), color='black', linestyle='--', label=f'Median: {food["poverty_rate"].median():.1f}%')
axes[0].legend()

# SNAP gap (eligible vs participants)
food['snap_gap'] = food['snap_eligible_est'] - food['snap_participants_est']
axes[1].hist(food['snap_gap'].dropna(), bins=50, color='#FF9800', edgecolor='white', alpha=0.8)
axes[1].set_xlabel('SNAP Gap (Eligible -- Participating)')
axes[1].set_ylabel('Number of Counties')
axes[1].set_title('SNAP Coverage Gap by County')

# No vehicle percentage
axes[2].hist(food['no_vehicle_pct'].dropna(), bins=50, color='#9C27B0', edgecolor='white', alpha=0.8)
axes[2].set_xlabel('Households Without Vehicle (%)')
axes[2].set_ylabel('Number of Counties')
axes[2].set_title('Vehicle Access Distribution')

plt.tight_layout()
plt.show()

print(f"\nMedian poverty rate: {food['poverty_rate'].median():.1f}%")
print(f"Counties with >25% poverty: {len(food[food['poverty_rate'] > 25]):,}")
print(f"Median no-vehicle rate: {food['no_vehicle_pct'].median():.1f}%")

## Housing Crisis -- Rent Burden Across America

What percentage of renters spend more than 30% or 50% of income on rent?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Rent burden distribution
axes[0].hist(housing['pct_rent_burdened_30plus'].dropna(), bins=50, color='#E91E63', edgecolor='white', alpha=0.8, label='30%+ of income')
axes[0].hist(housing['pct_rent_burdened_50plus'].dropna(), bins=50, color='#1565C0', edgecolor='white', alpha=0.6, label='50%+ of income')
axes[0].set_xlabel('Percent of Renters')
axes[0].set_ylabel('Number of Counties')
axes[0].set_title('Rent Burden Distribution')
axes[0].legend()

# Median rent vs median income scatter
ax2 = axes[1]
scatter = ax2.scatter(housing['median_household_income'].dropna() / 1000,
                      housing['median_gross_rent'].dropna(),
                      c=housing['pct_rent_burdened_30plus'].dropna(),
                      cmap='YlOrRd', s=5, alpha=0.5)
ax2.set_xlabel('Median Household Income ($K)')
ax2.set_ylabel('Median Gross Rent ($)')
ax2.set_title('Rent vs Income (colored by % rent-burdened)')
plt.colorbar(scatter, ax=ax2, label='% Rent Burdened (30%+)')

plt.tight_layout()
plt.show()

print(f"Median rent burden (30%+): {housing['pct_rent_burdened_30plus'].median():.1f}%")
print(f"Median rent burden (50%+): {housing['pct_rent_burdened_50plus'].median():.1f}%")
print(f"Median gross rent: ${housing['median_gross_rent'].median():,.0f}")
print(f"Median household income: ${housing['median_household_income'].median():,.0f}")

## Healthcare Deserts -- Uninsured and At-Risk Counties

County-level uninsured rates and hospital closure risk scores.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Uninsured rate distribution
axes[0].hist(healthcare['uninsured_rate'].dropna(), bins=50, color='#00BCD4', edgecolor='white', alpha=0.8)
axes[0].set_xlabel('Uninsured Rate (%)')
axes[0].set_ylabel('Number of Counties')
axes[0].set_title('County Uninsured Rate Distribution')
axes[0].axvline(healthcare['uninsured_rate'].median(), color='black', linestyle='--',
                label=f'Median: {healthcare["uninsured_rate"].median():.1f}%')
axes[0].legend()

# Risk category breakdown
risk_counts = healthcare['risk_category'].value_counts()
colors_risk = {'Low': '#4CAF50', 'Medium': '#FF9800', 'High': '#F44336', 'Critical': '#B71C1C'}
bar_colors = [colors_risk.get(cat, '#607D8B') for cat in risk_counts.index]
axes[1].bar(risk_counts.index, risk_counts.values, color=bar_colors)
axes[1].set_ylabel('Number of Counties')
axes[1].set_title('Hospital Closure Risk Category')
for i, (cat, val) in enumerate(risk_counts.items()):
    axes[1].text(i, val + 20, f'{val:,}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

# Rural vs urban
rural_counts = healthcare['rural_category'].value_counts()
print("\nRural/Urban breakdown:")
for cat, count in rural_counts.items():
    print(f"  {cat}: {count:,} counties ({100*count/len(healthcare):.1f}%)")

## Veterans and Firearms -- State-Level Analysis

Military service, veteran population, firearms ownership, and suicide rates by state.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Top 15 states by veteran percentage
top_vet = veterans.nlargest(15, 'veteran_percentage')
axes[0, 0].barh(range(len(top_vet)), top_vet['veteran_percentage'].values, color='#3F51B5')
axes[0, 0].set_yticks(range(len(top_vet)))
axes[0, 0].set_yticklabels(top_vet['NAME'].values)
axes[0, 0].set_xlabel('Veteran % of Population')
axes[0, 0].set_title('Top 15 States -- Veteran Percentage')
axes[0, 0].invert_yaxis()

# Firearms ownership
top_guns = veterans.dropna(subset=['ownership_percentage']).nlargest(15, 'ownership_percentage')
axes[0, 1].barh(range(len(top_guns)), top_guns['ownership_percentage'].values, color='#FF5722')
axes[0, 1].set_yticks(range(len(top_guns)))
axes[0, 1].set_yticklabels(top_guns['NAME'].values)
axes[0, 1].set_xlabel('Firearm Ownership (%)')
axes[0, 1].set_title('Top 15 States -- Firearm Ownership')
axes[0, 1].invert_yaxis()

# Veteran vs civilian suicide rate
valid_suicide = veterans.dropna(subset=['veteran_suicide_rate', 'civilian_suicide_rate'])
x = valid_suicide['civilian_suicide_rate']
y = valid_suicide['veteran_suicide_rate']
axes[1, 0].scatter(x, y, c='#E91E63', s=40, alpha=0.7)
axes[1, 0].plot([0, max(x.max(), y.max())], [0, max(x.max(), y.max())],
                'k--', alpha=0.3, label='Equal rate line')
axes[1, 0].set_xlabel('Civilian Suicide Rate')
axes[1, 0].set_ylabel('Veteran Suicide Rate')
axes[1, 0].set_title('Veteran vs Civilian Suicide Rates by State')
axes[1, 0].legend()

# PTSD prevalence
valid_ptsd = veterans.dropna(subset=['ptsd_prevalence_pct']).nlargest(15, 'ptsd_prevalence_pct')
axes[1, 1].barh(range(len(valid_ptsd)), valid_ptsd['ptsd_prevalence_pct'].values, color='#795548')
axes[1, 1].set_yticks(range(len(valid_ptsd)))
axes[1, 1].set_yticklabels(valid_ptsd['NAME'].values)
axes[1, 1].set_xlabel('PTSD Prevalence (%)')
axes[1, 1].set_title('Top States -- PTSD Prevalence Among Veterans')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.show()

vet_suicide_med = veterans['veteran_suicide_rate'].median()
civ_suicide_med = veterans['civilian_suicide_rate'].median()
print(f"Median veteran suicide rate: {vet_suicide_med:.1f}")
print(f"Median civilian suicide rate: {civ_suicide_med:.1f}")
print(f"Veteran risk ratio (median): {veterans['veteran_risk_ratio'].median():.2f}x")

## CMS Hospitals -- Facility Types and Ownership

5,400+ hospitals from the CMS Hospital Compare database.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Hospital type
type_counts = hospitals['hospital_type'].value_counts()
axes[0].barh(range(len(type_counts)), type_counts.values, color='#009688')
axes[0].set_yticks(range(len(type_counts)))
axes[0].set_yticklabels(type_counts.index)
axes[0].set_xlabel('Number of Hospitals')
axes[0].set_title('Hospitals by Type')
for i, val in enumerate(type_counts.values):
    axes[0].text(val + 10, i, f'{val:,}', va='center', fontsize=9)

# Ownership
own_counts = hospitals['hospital_ownership'].value_counts().head(8)
axes[1].barh(range(len(own_counts)), own_counts.values, color='#673AB7')
axes[1].set_yticks(range(len(own_counts)))
axes[1].set_yticklabels(own_counts.index)
axes[1].set_xlabel('Number of Hospitals')
axes[1].set_title('Hospitals by Ownership Type')
for i, val in enumerate(own_counts.values):
    axes[1].text(val + 10, i, f'{val:,}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

# Emergency services
emerg = hospitals['emergency_services'].value_counts()
print(f"\nEmergency services available: {emerg.get('Yes', 0):,} ({100*emerg.get('Yes',0)/len(hospitals):.1f}%)")
print(f"No emergency services: {emerg.get('No', 0):,} ({100*emerg.get('No',0)/len(hospitals):.1f}%)")

## Hospital Distribution by State

Which states have the most (and fewest) hospitals?

In [ ]:
state_counts = hospitals['state'].value_counts()

fig, ax = plt.subplots(figsize=(16, 7))
top_15 = state_counts.head(15)
bottom_10 = state_counts.tail(10)

ax.bar(range(len(top_15)), top_15.values, color='#2196F3', label='Top 15')
ax.set_xticks(range(len(top_15)))
ax.set_xticklabels(top_15.index)
ax.set_ylabel('Number of Hospitals')
ax.set_title('Hospitals by State -- Top 15')

for i, val in enumerate(top_15.values):
    ax.text(i, val + 5, str(val), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

print(f"\nTotal hospitals: {len(hospitals):,}")
print(f"States/territories: {hospitals['state'].nunique()}")
print(f"\nBottom 10 states:")
for state, count in bottom_10.items():
    print(f"  {state}: {count}")

## Cross-Dataset Analysis -- Poverty and Healthcare Access

Merging food desert and healthcare data to see how poverty relates to being uninsured.

In [ ]:
# Merge food desert poverty with healthcare uninsured
food['fips'] = food['fips'].astype(str).str.zfill(5)
healthcare['fips'] = healthcare['fips'].astype(str).str.zfill(5)

merged = food[['fips', 'poverty_rate', 'state']].merge(
    healthcare[['fips', 'uninsured_rate', 'risk_category']], on='fips', how='inner'
)

fig, ax = plt.subplots(figsize=(12, 8))
risk_colors = {'Low': '#4CAF50', 'Medium': '#FF9800', 'High': '#F44336', 'Critical': '#B71C1C'}
for risk, color in risk_colors.items():
    subset = merged[merged['risk_category'] == risk]
    if len(subset) > 0:
        ax.scatter(subset['poverty_rate'], subset['uninsured_rate'],
                   c=color, s=8, alpha=0.4, label=f'{risk} risk ({len(subset):,})')

ax.set_xlabel('Poverty Rate (%)')
ax.set_ylabel('Uninsured Rate (%)')
ax.set_title('Poverty Rate vs Uninsured Rate by County\n(colored by hospital closure risk)')
ax.legend(markerscale=3, loc='upper left')

# Add trend line
valid = merged.dropna(subset=['poverty_rate', 'uninsured_rate'])
z = np.polyfit(valid['poverty_rate'], valid['uninsured_rate'], 1)
p = np.poly1d(z)
x_line = np.linspace(0, valid['poverty_rate'].max(), 100)
ax.plot(x_line, p(x_line), 'k--', alpha=0.5, linewidth=2, label=f'Trend (r={np.corrcoef(valid["poverty_rate"], valid["uninsured_rate"])[0,1]:.2f})')
ax.legend(markerscale=3, loc='upper left')

plt.tight_layout()
plt.show()

corr = valid['poverty_rate'].corr(valid['uninsured_rate'])
print(f"Correlation between poverty and uninsured rate: {corr:.3f}")
print(f"Counties merged: {len(merged):,}")

## Summary Statistics

In [ ]:
print("=" * 60)
print("US INEQUALITY ATLAS -- SUMMARY")
print("=" * 60)
print(f"Counties (food deserts):     {len(food):>8,}")
print(f"Counties (housing):          {len(housing):>8,}")
print(f"Counties (healthcare):       {len(healthcare):>8,}")
print(f"States (veterans):           {len(veterans):>8,}")
print(f"CMS hospitals:               {len(hospitals):>8,}")
print()
print("Key findings:")
print(f"  Median poverty rate:       {food['poverty_rate'].median():.1f}%")
print(f"  Median uninsured rate:     {healthcare['uninsured_rate'].median():.1f}%")
print(f"  Median rent burden (30%+): {housing['pct_rent_burdened_30plus'].median():.1f}%")
print(f"  Median rent burden (50%+): {housing['pct_rent_burdened_50plus'].median():.1f}%")
print(f"  Veteran suicide risk ratio:{veterans['veteran_risk_ratio'].median():.2f}x civilian")
print(f"  Hospitals w/ emergency:    {100*hospitals[hospitals['emergency_services']=='Yes'].shape[0]/len(hospitals):.1f}%")
high_risk = healthcare[healthcare['risk_category'].isin(['High', 'Critical'])]
print(f"  High/Critical risk counties: {len(high_risk):,} ({100*len(high_risk)/len(healthcare):.1f}%)")
print("=" * 60)